In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
!pip install git+https://github.com/afnan47/cuda.git
%load_ext nvcc_plugin

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-n3zh33x_
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-n3zh33x_
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=6e691a58decbaa969df62c5ddf535ec381f7990cbb57aa5b91ec8b6d4cb0d1f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-qz814qmw/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


# **Vector Addition**

In [22]:
%%cu
#include "stdio.h"
#include <iostream>
#include <cuda.h>
#include <cuda_runtime.h>

#define N 5


__global__ void gpuAdd(int *d_a, int *d_b, float *d_result)
{
    int tid = blockIdx.x;
    if (tid < N)
        d_result[tid] = sqrt((float)((d_a[tid] * d_a[tid] )+ (d_b[tid] * d_b[tid])));
}

int main(void)
{

    int h_a[N], h_b[N];
    float h_c[N];

    int *d_a, *d_b;
    float *d_c;

    cudaMalloc((void **)&d_a, N * sizeof(int));
    cudaMalloc((void **)&d_b, N * sizeof(int));
    cudaMalloc((void **)&d_c, N * sizeof(float));

    for (int i = 0; i < N; i++) {
        h_a[i] = 2*i*i;
        h_b[i] = i;
    }

    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);
    gpuAdd <<<N, 256 >>>(d_a, d_b, d_c);
    // Copy result back to host memory from device memory
    cudaMemcpy(h_c, d_c, N * sizeof(float), cudaMemcpyDeviceToHost);
    printf("Vector addition on GPU \n");
    for (int i = 0; i < N; i++) {
        printf("The sum of %d element is %d + %d = %f\n",
            i, h_a[i], h_b[i],h_c[i]);
    }
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}

Vector addition on GPU 
The sum of 0 element is 0 + 0 = 0.000000
The sum of 1 element is 2 + 1 = 2.236068
The sum of 2 element is 8 + 2 = 8.246211
The sum of 3 element is 18 + 3 = 18.248287
The sum of 4 element is 32 + 4 = 32.249031



In [ ]:
%%cu
#include "stdio.h"
#include <iostream>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>


__global__ void gpuAdd(int *d_a, int *d_b, float *d_result,int n)
{
    int tid = blockIdx.x;
    if (tid < n)
        d_result[tid] = sqrt((float)(d_a[tid] * d_a[tid] + d_b[tid] * d_b[tid]));
}

int main(void)
{
    int N;
    std::cout << "Enter the size of vector: ";
    std::cin >> N;
    int h_a[N], h_b[N];
    float h_c[N];
    // Defining device pointers
    int *d_a, *d_b;
    float *d_c;

    cudaMalloc((void **)&d_a, N * sizeof(int));
    cudaMalloc((void **)&d_b, N * sizeof(int));
    cudaMalloc((void **)&d_c, N * sizeof(float));
    // Initializing Arrays
    for (int i = 0; i < N; i++) {
        std::cout << "Enter the value of element " << i << " of array h_a: ";
        std::cin >> h_a[i];
        std::cout << "Enter the value of element " << i << " of array h_b: ";
        std::cin >> h_b[i];
    }

    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);


    gpuAdd<<<N , 256>>>(d_a, d_b, d_c, N);

    cudaMemcpy(h_c, d_c, N * sizeof(float), cudaMemcpyDeviceToHost);
    printf("Vector addition on GPU \n");


    for (int i = 0; i < N; i++) {
        printf("The sum of %d element is %d + %d = %f\n",
            i, h_a[i], h_b[i],h_c[i]);
    }

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}



# **Matrix Multiplication**

In [23]:
%%cu
#include <iostream>
#include <cuda.h>

using namespace std;

#define BLOCK_SIZE 2

__global__ void gpuMM(float *A, float *B, float *C, int N)
{

    int row = blockIdx.y*blockDim.y + threadIdx.y;
    int col = blockIdx.x*blockDim.x + threadIdx.x;

    float sum = 0.f;
    for (int n = 0; n < N; ++n)
        sum += A[row*N+n]*B[n*N+col];

    C[row*N+col] = sum;
}

int main(int argc, char *argv[])
{int N;float K;

	cout<<"Enter a Value for Size/2 of matrix";
	cin>>K;

    K = 1;
    N = K*BLOCK_SIZE;

    cout << "\n Executing Matrix Multiplcation" << endl;
    cout << "\n Matrix size: " << N << "x" << N << endl;


    float *hA,*hB,*hC;
    hA = new float[N*N];
    hB = new float[N*N];
    hC = new float[N*N];


    for (int j=0; j<N; j++){
        for (int i=0; i<N; i++){
           hA[j*N+i] = 2;
           hB[j*N+i] = 4;

        }
    }


    int size = N*N*sizeof(float);
    float *dA,*dB,*dC;
    cudaMalloc(&dA,size);
    cudaMalloc(&dB,size);
    cudaMalloc(&dC,size);

    dim3 threadBlock(BLOCK_SIZE,BLOCK_SIZE);
    dim3 grid(K,K);
    cout<<"\n Input Matrix 1 \n";
    for (int row=0; row<N; row++){
            for (int col=0; col<N; col++){

                   cout<<hA[row*col]<<"	";

            }
            cout<<endl;
        }
    cout<<"\n Input Matrix 2 \n";
    for (int row=0; row<N; row++){
            for (int col=0; col<N; col++){

                   cout<<hB[row*col]<<"	";

            }
            cout<<endl;
        }

    cudaMemcpy(dA,hA,size,cudaMemcpyHostToDevice);
    cudaMemcpy(dB,hB,size,cudaMemcpyHostToDevice);


    gpuMM<<<grid,threadBlock>>>(dA,dB,dC,N);

    // Now do the matrix multiplication on the CPU
   /*float sum;
    for (int row=0; row<N; row++){
        for (int col=0; col<N; col++){
            sum = 0.f;
            for (int n=0; n<N; n++){
                sum += hA[row*N+n]*hB[n*N+col];
            }
            hC[row*N+col] = sum;
            cout << sum <<"	";


        }
        cout<<endl;
    }*/


    float *C;
    C = new float[N*N];


    cudaMemcpy(C,dC,size,cudaMemcpyDeviceToHost);

    cout <<"\n\n\n\n\n Resultant matrix\n\n";
    for (int row=0; row<N; row++){
        for (int col=0; col<N; col++){

               cout<<C[row*col]<<"	";

        }
        cout<<endl;
    }

    cout << "Finished." << endl;
}

Enter a Value for Size/2 of matrix
 Executing Matrix Multiplcation

 Matrix size: 2x2

 Input Matrix 1 
2	2	
2	2	

 Input Matrix 2 
4	4	
4	4	





 Resultant matrix

16	16	
16	16	
Finished.

